In [4]:
import requests 
import open3d as o3d
import os

In [5]:
os.environ['http_proxy'] = "http://proxy.robotron.de:80" 
os.environ['https_proxy'] = "https://proxy.robotron.de:80" 

In [6]:
url = "https://rcv.westeurope.cloudapp.azure.com/3dodserving/api/models"

payload = {}
files = {}
headers = {
  'Content-Type': 'multipart/form-data; boundary=--------------------------555221334256103757881987'
}

response = requests.request("GET", url, headers=headers, data = payload, files = files)

print(response.text.encode('utf8'))

b'[{"id":"1","name":"box3d","type":"3d_detection"}]'


In [12]:
url = "https://rcv.westeurope.cloudapp.azure.com/3dodserving/api/detect?model_id=1"

files = {'file': open('0005.ply','rb')}

response = requests.post(url,  files = files)

response.json()

[{'class_id': 0,
  'coordinates': {'0': [0.3450233464789509,
    0.01635452546924354,
    1.9177392442230334],
   '1': [0.543365186289017, 0.01635452546924354, 1.3817700181431343],
   '2': [-0.12132008128882638, 0.01635452546924354, 1.1357952635284314],
   '3': [-0.3196619210988924, 0.01635452546924354, 1.6717644896083306],
   '4': [0.3450233464789509, -0.527928257368505, 1.9177392442230334],
   '5': [0.543365186289017, -0.527928257368505, 1.3817700181431343],
   '6': [-0.12132008128882638, -0.527928257368505, 1.1357952635284314],
   '7': [-0.3196619210988924, -0.527928257368505, 1.6717644896083306]},
  'probability': 0.9834211468696594}]

In [13]:
print('class: ' + str(response.json()[0]['class_id']))
print('confidence: ' + str(response.json()[0]['probability']))

class: 0
confidence: 0.9834211468696594


In [35]:
points = []
for i in range(0,8):
    points.append(response.json()[0]['coordinates'][str(i)])
    i +=1

lines = [
        [0, 1],
        [0, 3],
        [1, 5],
        [3, 7],
        [4, 0],
        [4, 5],
        [5, 6],
        [6, 7],
        [2, 3],
        [2, 6],
        [2, 1],
        [4, 7]
        ]

colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.rotate(line_set.get_rotation_matrix_from_xyz([300,0,0]), center=False)
line_set.colors = o3d.utility.Vector3dVector(colors)

pcd = o3d.io.read_point_cloud('0005_cl.ply')
o3d.visualization.draw_geometries([line_set, pcd])    